In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## CNN model define

In [59]:
class Model:
    def __init__(self, sess, name, learning_rate=0.01):
        self.sess=sess
        self.name=name
        self.learning_rate = learning_rate
        self.training = False
        self._build_net()
    
    def _build_net(self):
        # placeholder
        self.x = tf.placeholder(tf.float32, [None, 784])
        x_img = tf.reshape(self.x, [-1,28,28,1])
        self.y = tf.placeholder(tf.float32, [None, 10])

        # Layer1 input -> ?, 28,28,1
        l1 = tf.layers.conv2d(inputs = x_img, filters=32, kernel_size = [3,3], strides=[1,1], padding="SAME", 
    activation=tf.nn.relu, kernel_initializer = tf.contrib.layers.xavier_initializer(), bias_initializer = tf.contrib.layers.xavier_initializer())
        l1 = tf.layers.max_pooling2d(inputs=l1, pool_size = [2,2], strides=2, padding="SAME")
        l1 = tf.layers.dropout(inputs=l1, rate=0.7, training = self.training)
        """
        w1 = tf.get_variable("w1", shape=[3,3,1,32], initializer=tf.contrib.layers.xavier_initializer())
        b1 = tf.get_variable("b1", shape=[32], initializer = tf.contrib.layers.xavier_initializer())
        l1 = tf.nn.convd2(x_img, w1, strides = [1,1,1,1], padding ="SAME") + b1
        l1 = tf.nn.relu(l1)
        l1 = tf.nn.max_pool(l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding = "SAME")
        l1 = tf.nn.dropout(l1, keep_prob=keep_prob)"""
        # Layer2 input-> ?,14,14,32
        l2 = tf.layers.conv2d(inputs=l1, filters=64, kernel_size=[3,3], strides=[1,1], padding="SAME", activation = tf.nn.relu,
    kernel_initializer = tf.contrib.layers.xavier_initializer(), bias_initializer = tf.contrib.layers.xavier_initializer())
        l2 = tf.layers.max_pooling2d(inputs=l2, pool_size=[2,2], strides= 2, padding="SAME")
        l2 = tf.layers.dropout(inputs=l2, rate=0.7, training=self.training)
        """
        w2 = tf.get_variable("w2", [3,3,32,64], initializer = tf.contrib.layers.xavier_initializer())
        b2 = tf.get_variable("b2", [64], initializer = tf.contrib.layers.xavier_initializer())
        l2 = tf.nn.conv2d(l1, w2, strides = [1,1,1,1], padding="SAME") + b
        l2 = tf.nn.relu(l2)
        l2 = tf.nn.pool(l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        l2 = tf.nn.dropout(l2, keep_prob = keep_prob)"""
        # Layer3 input -> ?, 7, 7, 64
        l3 = tf.layers.conv2d(inputs=l2, filters=128, kernel_size=[3,3],strides=[1,1], padding="SAME", activation = tf.nn.relu,
            kernel_initializer = tf.contrib.layers.xavier_initializer(), bias_initializer = tf.contrib.layers.xavier_initializer())
        l3 = tf.layers.max_pooling2d(inputs = l3, pool_size = [2,2],strides=2, padding="SAME")
        l3 = tf.layers.dropout(inputs=l3, rate=0.7, training=self.training)
        l3 = tf.reshape(l3, [-1, 4*4*128])
        # Layer4 input -> ?, 4, 4, 128
        l4 = tf.layers.dense(inputs=l3, units=600, activation = tf.nn.relu,
            kernel_initializer = tf.contrib.layers.xavier_initializer(), bias_initializer = tf.contrib.layers.xavier_initializer())
        l4 = tf.layers.dropout(inputs=l4, rate=0.7, training = self.training)
        """
        w4 = tf.get_variable("w4", [4*4*128, 600], initializer=tf.contrib.layers.xavier_initializer())
        b4 = tf.get_variable("b4", [600], initializer=tf.contrib.layers.xavier_initializer())
        l4 = tf.nn.relu(tf.matmul(l3, w4)+b4)
        l4 = tf.nn.dropout(l4, keep_prob = keep_prob)
        """
        # Layer5 input -> -1, 600
        self.hypothesis = tf.layers.dense(l4, units=10, activation = tf.nn.softmax,
     kernel_initializer = tf.contrib.layers.xavier_initializer(), bias_initializer = tf.contrib.layers.xavier_initializer()) 
        # dropout은 가중치 변수를 줄이기 위해서 하는 건데 더이상 가중치 곱이 없기 때문에 할 필요 X
        """
        w5 = tf.get_variable("w5", [600,10], initializer = tf.contrib.layers.xavier_initializer())
        b5 = tf.get_variable("b5", [10], initializer = tf.contrib.layers.xavier_initializer())"""
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=tf.layers.dense(l4,units=10, kernel_initializer=tf.contrib.layers.xavier_initializer(),bias_initializer=tf.contrib.layers.xavier_initializer()), labels = self.y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        self.predicted = tf.argmax(self.hypothesis,1)
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(self.predicted, tf.argmax(self.y,1)),dtype=tf.float32))
    
    def predict(self, x_test, *y_test):
        return self.sess.run(self.hypothesis, feed_dict={self.x:x_test})
    
    def get_accuracy(self,x_test,y_test):
        return self.sess.run(self.accuracy, feed_dict={self.x:x_test, self.y:y_test})
    def train(self, x_train, y_train):
        self.training = True
        return self.sess.run([self.cost,self.optimizer], feed_dict={self.x:x_train,self.y:y_train})
        
   
 

## Ensemble

In [60]:
sess=tf.Session()
num_emsemble = int(input())
models = [Model(sess,"mmmmmmmmmmmm"+str(m)) for m in range(num_emsemble)]

6


## CNN model train with Ensemble

In [54]:
#parameters
training_epochs = 15
batch_size = 100
num = mnist.train.num_examples//batch_size

sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_c = [0 for i in range(len(models))]
    for step in range(num):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs,batch_ys)
    avg_c[m_idx]+= c/num
    print("Epoch : %4d"%(epoch+1), f"Cost : {avg_c}")

Epoch :    1 Cost : [2.6071498513221734, 1.54443053483963, 1.2018889643251909, 2.283391987681389, 0.9246945463120939, 1.792877428829669]
Epoch :    2 Cost : [2.30303397655487, 0.24557689651846887, 0.19033064190298324, 0.5338413144648074, 0.17390169583261011, 0.276357107758522]
Epoch :    3 Cost : [2.302898774147035, 0.17221074588596819, 0.1420222579687834, 0.25009446300566196, 0.11710533281788231, 0.1724373467639089]
Epoch :    4 Cost : [2.303658018112183, 0.1329263753816486, 0.10978900396265087, 0.15993022479116917, 0.09810040744021535, 0.14062123492360112]
Epoch :    5 Cost : [2.3031508302688586, 0.11517988426610827, 0.1006774523667991, 0.14008260086178786, 0.10487858206033708, 0.12701854706741872]
Epoch :    6 Cost : [2.3024141025543208, 0.10257270011119542, 0.08828347455710174, 0.12404754523187878, 0.08906401371583338, 0.1114277506060898]
Epoch :    7 Cost : [2.303234069347382, 0.10963509653694924, 0.08710013121366503, 0.10889425227418545, 0.08516605015844109, 0.10129618924111128]


## Ensemble prediction using sum

In [58]:
test_size = len(mnist.test.labels)
print(test_size)
predictions = np.zeros(test_size*10).reshape(-1,10)

for m_idx, m in enumerate(models):
    print(f"{m_idx} model accuracy :", m.get_accuracy(mnist.test.images, mnist.test.labels))
    p=m.hypothesis(feed_dict={m.x:mnist.test.images})
    predictions +=p
    
ensemble_correct_prediction = tf.equal(tf.argmax(predictions,1),tf.argmax(mnist.test.labels,1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction,dtype=tf.float32))

print("Ensemble accuracy" , sess.run(ensemble_accuracy))

10000
0 model accuracy : 0.0892


TypeError: 'Tensor' object is not callable

In [ ]:
# Deep & Wide?
# CIFAR 10
# imagenet